In [1]:
install.packages("fBasics")
install.packages("xgboost")
install.packages("randomForest")
library(fBasics)
library(xgboost)
library(randomForest)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘timeDate’, ‘timeSeries’, ‘gss’, ‘stabledist’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.



In [2]:
install.packages("ramify")
library(ramify)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘ramify’


The following objects are masked from ‘package:fBasics’:

    inv, tr


The following object is masked from ‘package:graphics’:

    clip




# Вариант 16

### Обучение и проверка точности

In [54]:
Train <- read.csv("train_var_16.csv",header = TRUE,sep =",")

In [55]:
MultiLogLoss <- function(y,p)
{
  nrow <- dim(y)[1]
  ncol <- dim(y)[2]
  s <- NA
  p0 <- ifelse(p != 0,p,1)
  if ((nrow == dim(p)[1]) &&  (ncol == dim(p)[2]))
  {
     s <- -sum(y * log(p0))/nrow
  }
  return(s)
}

In [56]:
set.seed(13)
testInd = sample(nrow(Train), nrow(Train)/3)
Train$f <- as.factor(Train$f)

xTrain = Train[-testInd,]

xTest = Train[testInd,]
yTrain = as.factor(Train$f[-testInd])
yTest = Train$f[testInd]

**Random Forest**

In [5]:
rfOtto <- randomForest(f~., data=xTrain,ntree=100)
rfPred <- predict(rfOtto, xTest, type="prob")
rfPred[1:5,]

,A,B,C,D
259,0.13,0.03,0.83,0.01
74,0.01,0.28,0.08,0.63
205,0.02,0.00,0.23,0.75
262,0.00,0.91,0.04,0.05
132,0.00,0.58,0.09,0.33


In [6]:
rfPredTest <- predict(rfOtto, xTest, type="response")
head(rfPredTest)

259  74 205 262 132 221 
  C   D   D   B   B   A 
Levels: A B C D

In [ ]:
print(rfPredTest==yTest)  #21

  [1]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE FALSE  TRUE
 [13]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [25] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE
 [37]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE FALSE  TRUE FALSE
 [49] FALSE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE  TRUE
 [61]  TRUE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
 [73]  TRUE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
 [85]  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
 [97]  TRUE FALSE  TRUE  TRUE


In [7]:
# самое идиотское на свете создание матрицы, потому что библиотека не подключается
A <- c()
B <- c()
C <- c()
D <- c()
for(i in 1:length(yTest)) {
    A[i] <- 'A'
    B[i] <- 'B'
    C[i] <- 'C'
    D[i] <- 'D'
}
yTest_A <- as.numeric(yTest == A)
yTest_B <- as.numeric(yTest == B)
yTest_C <- as.numeric(yTest == C)
yTest_D <- as.numeric(yTest == D)
rf_target_IndMat <- cbind(yTest_A, yTest_B, yTest_C, yTest_D)
rf_target_IndMat[1:10,]

yTest_A,yTest_B,yTest_C,yTest_D
0,0,1,0
0,0,0,1
0,0,1,0
0,1,0,0
0,1,0,0
1,0,0,0
1,0,0,0
0,0,0,1
0,1,0,0
0,0,1,0


In [10]:
(rfLogLoss = MultiLogLoss(rf_target_IndMat,rfPred))

[1] 0.5172123

Чё-то капец оно большое.....

**Gradient boosting**

In [69]:
set.seed(13)
testInd = sample(nrow(Train), nrow(Train)/3)
Train$f <- as.factor(Train$f)

xTrain = Train[-testInd,]

xTest = Train[testInd,]
yTrain = as.factor(Train$f[-testInd])
yTest = Train$f[testInd]

In [70]:
xTrain = data.matrix(xTrain[c(1, 2, 3)])
xTest = data.matrix(xTest[c(1, 2, 3)])
yTrain = as.integer(yTrain)-1
table(yTrain)

yTrain
 0  1  2  3 
37 40 63 60 

In [71]:
numClasses = 4
param <- list("objective" = "multi:softprob",
              "eval_metric" = "mlogloss",
              "num_class" = numClasses)

In [109]:
cv.nround <- 67 # подбором, вроде это оптимальный варик
cv.nfold <- 6
set.seed(1)
bst.cv = xgb.cv(param=param, data = xTrain, label = yTrain,
                nfold = cv.nfold, nrounds = cv.nround,verbose=F)
bst = xgboost(param=param, data = xTrain, label = yTrain,
              nrounds=cv.nround,verbose=F)
xgbPred <- matrix(predict(bst, xTest), ncol = numClasses, byrow = TRUE)
head(xgbPred)

3.467708e-02,1.482020e-03,9.597296e-01,0.0041112802
1.093910e-03,2.813158e-02,2.216364e-03,0.9685580730
7.206434e-04,8.526382e-04,1.287041e-01,0.8697225451
4.229703e-05,9.994122e-01,6.860662e-05,0.0004768874
1.004494e-03,7.143434e-01,2.244286e-03,0.2824078202
9.986081e-01,8.442687e-05,1.135516e-03,0.0001720077


In [110]:
(gbLogLoss = MultiLogLoss(rf_target_IndMat,xgbPred))

[1] 0.3970963

In [111]:
xgbPredTestNum = argmax(xgbPred)
xgbPredTest = c()
for(i in 1:length(xgbPredTestNum)) {
    if (xgbPredTestNum[i] == 1) {
        xgbPredTest[i] <- 'A'
    } else {
        if (xgbPredTestNum[i] == 2) {
            xgbPredTest[i] <- 'B'
        } else {
            if (xgbPredTestNum[i] == 3) {
                xgbPredTest[i] <- 'C'
            } else {
                xgbPredTest[i] <- 'D'
            }
        }
    }
}
print(xgbPredTest==yTest)  #16

  [1]  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [13]  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [25]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [37]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE
 [49]  TRUE  TRUE FALSE FALSE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE
 [61]  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE FALSE  TRUE FALSE  TRUE
 [73]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE
 [85]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
 [97]  TRUE FALSE  TRUE  TRUE


По MultiLogLoss и если посчитать втупую количество FALSE в каждом предсказании, видно, что лучше всего работает xgb при cv.nround=40

### Получение результатов

In [62]:
Train <- read.csv("train_var_16.csv",header = TRUE,sep =",")
Train$f <- as.factor(Train$f)
xTrain = Train[c(1, 2, 3)]
yTrain = Train$f
xTest <- read.csv("test_var_16.csv",header = TRUE,sep =",")
head(xTrain,10)

,x1,x2,x3
,<dbl>,<dbl>,<dbl>
1,0.740582499,1.46121844,0.65099881
2,0.336409246,0.42546037,2.38805983
3,0.193430599,0.70941501,1.97439871
4,0.280807660,0.06937908,2.84477286
5,0.997864848,1.65404679,0.04714673
6,0.534172288,1.70515807,0.91571973
7,0.958553967,1.69493166,1.83806391
8,0.003312868,1.64359377,0.58816586
9,0.417490384,1.08938486,1.50861648


In [63]:
xTrain = data.matrix(xTrain)
xTest = data.matrix(xTest)
yTrain = as.integer(yTrain)-1
table(yTrain)

yTrain
 0  1  2  3 
60 56 93 91 

In [64]:
xTest

x1,x2,x3
0.941198268,0.35511204,0.45254143
0.814925580,1.38527870,0.27652185
0.453266963,0.83678957,2.31972561
0.996952858,1.82794465,0.47186478
0.346013498,0.02943333,0.87362706
0.922335447,0.51821177,0.37794253
0.084155077,1.68719154,1.87151217
0.187454387,1.57806332,2.71118174
0.305729876,1.21940248,0.23187057
0.505229047,0.31250447,0.39984412


In [65]:
numClasses = 4
param <- list("objective" = "multi:softprob",
              "eval_metric" = "mlogloss",
              "num_class" = numClasses)

In [112]:
cv.nround <- 67
cv.nfold <- 6
set.seed(1)
bst.cv = xgb.cv(param=param, data = xTrain, label = yTrain,
                nfold = cv.nfold, nrounds = cv.nround,verbose=F)
bst = xgboost(param=param, data = xTrain, label = yTrain,
              nrounds=cv.nround,verbose=F)
xgbPred <- matrix(predict(bst, xTest), ncol = numClasses, byrow = TRUE)
head(xgbPred)

3.467708e-02,1.482020e-03,9.597296e-01,0.0041112802
1.093910e-03,2.813158e-02,2.216364e-03,0.9685580730
7.206434e-04,8.526382e-04,1.287041e-01,0.8697225451
4.229703e-05,9.994122e-01,6.860662e-05,0.0004768874
1.004494e-03,7.143434e-01,2.244286e-03,0.2824078202
9.986081e-01,8.442687e-05,1.135516e-03,0.0001720077


In [113]:
xgbPredTestNum = argmax(xgbPred)
xgbPredTest = c()
for(i in 1:length(xgbPredTestNum)) {
    if (xgbPredTestNum[i] == 1) {
        xgbPredTest[i] <- 'A'
    } else {
        if (xgbPredTestNum[i] == 2) {
            xgbPredTest[i] <- 'B'
        } else {
            if (xgbPredTestNum[i] == 3) {
                xgbPredTest[i] <- 'C'
            } else {
                xgbPredTest[i] <- 'D'
            }
        }
    }
}
xgbPredTest

[1] "C" "D" "D" "B" "B" "A" "A" "D" "B" "C" "D" "A" "B" "C" "D" "C" "D" "C"
 [19] "D" "C" "B" "C" "C" "A" "C" "A" "D" "D" "D" "C" "D" "D" "D" "A" "B" "D"
 [37] "D" "A" "A" "D" "A" "D" "C" "D" "B" "C" "D" "B" "D" "A" "B" "A" "C" "C"
 [55] "D" "D" "D" "C" "B" "B" "A" "A" "A" "D" "C" "C" "A" "D" "C" "D" "C" "C"
 [73] "B" "C" "D" "D" "C" "C" "D" "B" "D" "C" "C" "D" "D" "C" "C" "A" "D" "A"
 [91] "C" "B" "B" "D" "C" "A" "A" "C" "A" "B"

In [114]:
write.csv(xgbPredTest, 'result.csv', row.names = FALSE)